In [1]:
import os
import pandas as pd
import numpy as np
import collections
from sklearn.preprocessing import StandardScaler


수비점수 = 수비율 * 수비이닝 * 포지션 가중치
March
포지션 가중치
- 포수 = 8
- 내야수
  = 1루수 2
  = 3루수 4
  = 2루수 6
  = 유격수 7
- 외야수
  = 좌익수 2
  = 우익수 2.5
  = 중견수 3.5
  
kWAR 포지션보정점수 
MinMaxScaler
포수 = 1.0     ## 1.0
1루수 = 0.0    ## -1.0
2루수 = 0.6    ## 0.2
3루수 = 0.6    ## 0.2
유격수 = 0.8   ## 0.6
좌익수 = 0.2   ## -0.6
중견수 = 0.4   ## -0.2
우익수 = 0.2   ## -0.6
  
FPCT (수비율or 수비성공률) - (자살+보살) / (자살+보살+실책) 

In [40]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율')
dfn_1 = pd.read_csv('2019_전체.csv')
dfn_1.index = dfn_1['선수명']
del dfn_1['선수명']
dfn_1 = dfn_1.drop(columns = ['G', 'GS', 'PKO', 'PO', 'A', 'DP', 'PB', 'SB', 'CS', 'CS%'])
dfn_1 = dfn_1.replace('-', '0')
dfn_1

,팀명,POS,IP,E,FPCT
선수명,,,,,
강경학,한화,2루수,51.00,1,0.971
강경학,한화,1루수,69.00,0,1.000
강경학,한화,유격수,153.00,3,0.965
강로한,롯데,3루수,183.33,3,0.950
강로한,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000
황재균,KT,유격수,54.00,1,0.968
황재균,KT,3루수,964.00,10,0.970


In [41]:
## integer로 타입변환
FPCT_list = []
for a in dfn_1['FPCT'] :
    FPCT_list.append(int(a.replace('.', '').strip()) / 1000)
dfn_1['FPCT'] = FPCT_list
dfn_1

,팀명,POS,IP,E,FPCT
선수명,,,,,
강경학,한화,2루수,51.00,1,0.971
강경학,한화,1루수,69.00,0,1.000
강경학,한화,유격수,153.00,3,0.965
강로한,롯데,3루수,183.33,3,0.950
강로한,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000
황재균,KT,유격수,54.00,1,0.968
황재균,KT,3루수,964.00,10,0.970


In [42]:
## 이닝 표준화
#StandardScaler().fit_transform(df_fin)
dfn_11 = pd.DataFrame(dfn_1['IP'])
dfn_11
dfn_12 = pd.DataFrame(StandardScaler().fit_transform(dfn_11), index = dfn_1.index, columns = ['s_IP'])
dfn_1['IP'] = dfn_12['s_IP']
dfn_1

,팀명,POS,IP,E,FPCT
선수명,,,,,
강경학,한화,2루수,-0.429937,1,0.971
강경학,한화,1루수,-0.364028,0,1.000
강경학,한화,유격수,-0.056452,3,0.965
강로한,롯데,3루수,0.054605,3,0.950
강로한,롯데,유격수,0.341456,12,0.911
...,...,...,...,...,...
황재균,KT,1루수,-0.584933,0,1.000
황재균,KT,유격수,-0.418952,1,0.968
황재균,KT,3루수,2.913118,10,0.970


In [38]:
#dfn_1['s_IP'] = dfn_12['s_IP']
#dfn_1

,팀명,POS,IP,E,FPCT,s_IP
선수명,,,,,,
강경학,한화,2루수,51.00,1,0.971,-0.429937
강경학,한화,1루수,69.00,0,1.000,-0.364028
강경학,한화,유격수,153.00,3,0.965,-0.056452
강로한,롯데,3루수,183.33,3,0.950,0.054605
강로한,롯데,유격수,261.67,12,0.911,0.341456
...,...,...,...,...,...,...
황재균,KT,1루수,8.67,0,1.000,-0.584933
황재균,KT,유격수,54.00,1,0.968,-0.418952
황재균,KT,3루수,964.00,10,0.970,2.913118


#### March score

In [43]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_1)) :
    if dfn_1.iloc[a, 1] == '1루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '3루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 4, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '2루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 6, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '유격수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 7, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '좌익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '중견수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 3.5, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '우익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 2.5, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '포수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 8, 3)
        D_score_list.append(D_score)
dfn_1['March_score'] = D_score_list
dfn_1

,팀명,POS,IP,E,FPCT,March_score
선수명,,,,,,
강경학,한화,2루수,-0.429937,1,0.971,-2.505
강경학,한화,1루수,-0.364028,0,1.000,-0.728
강경학,한화,유격수,-0.056452,3,0.965,-0.381
강로한,롯데,3루수,0.054605,3,0.950,0.207
강로한,롯데,유격수,0.341456,12,0.911,2.177
...,...,...,...,...,...,...
황재균,KT,1루수,-0.584933,0,1.000,-1.170
황재균,KT,유격수,-0.418952,1,0.968,-2.839
황재균,KT,3루수,2.913118,10,0.970,11.303


#### kWAR 포지션 보정점수 기준

In [44]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_1)) :
    if dfn_1.iloc[a, 1] == '1루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '3루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '2루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '유격수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.8, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '좌익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '중견수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.4, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '우익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '포수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 1.0, 3)
        D_score_list.append(D_score)
dfn_1['kWAR_score'] = D_score_list
dfn_1

,팀명,POS,IP,E,FPCT,March_score,kWAR_score
선수명,,,,,,,
강경학,한화,2루수,-0.429937,1,0.971,-2.505,-0.250
강경학,한화,1루수,-0.364028,0,1.000,-0.728,-0.000
강경학,한화,유격수,-0.056452,3,0.965,-0.381,-0.044
강로한,롯데,3루수,0.054605,3,0.950,0.207,0.031
강로한,롯데,유격수,0.341456,12,0.911,2.177,0.249
...,...,...,...,...,...,...,...
황재균,KT,1루수,-0.584933,0,1.000,-1.170,-0.000
황재균,KT,유격수,-0.418952,1,0.968,-2.839,-0.324
황재균,KT,3루수,2.913118,10,0.970,11.303,1.695


#### AHP 결과 계수

In [45]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_1)) :
    if dfn_1.iloc[a, 1] == '1루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.022, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '3루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.124, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '2루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.124, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '유격수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.219, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '좌익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.036, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '중견수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.067, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '우익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.036, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '포수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.370, 3)
        D_score_list.append(D_score)
dfn_1['AHP_score'] = D_score_list
dfn_1

,팀명,POS,IP,E,FPCT,March_score,kWAR_score,AHP_score
선수명,,,,,,,,
강경학,한화,2루수,-0.429937,1,0.971,-2.505,-0.250,-0.052
강경학,한화,1루수,-0.364028,0,1.000,-0.728,-0.000,-0.008
강경학,한화,유격수,-0.056452,3,0.965,-0.381,-0.044,-0.012
강로한,롯데,3루수,0.054605,3,0.950,0.207,0.031,0.006
강로한,롯데,유격수,0.341456,12,0.911,2.177,0.249,0.068
...,...,...,...,...,...,...,...,...
황재균,KT,1루수,-0.584933,0,1.000,-1.170,-0.000,-0.013
황재균,KT,유격수,-0.418952,1,0.968,-2.839,-0.324,-0.089
황재균,KT,3루수,2.913118,10,0.970,11.303,1.695,0.350


In [46]:
dfn_1_uniq = dfn_1.reset_index()
dfn_1_uniq = dfn_1_uniq.append({'선수명' : 0},  ignore_index = True) ## 루프 돌릴 때 마지막 row 하나 더 필요해서 추가
dfn_1_uniq = dfn_1_uniq.drop(columns = ['팀명', 'POS'])
dfn_1_uniq

,선수명,IP,E,FPCT,March_score,kWAR_score,AHP_score
0,강경학,-0.429937,1.0,0.971,-2.505,-0.250,-0.052
1,강경학,-0.364028,0.0,1.000,-0.728,-0.000,-0.008
2,강경학,-0.056452,3.0,0.965,-0.381,-0.044,-0.012
3,강로한,0.054605,3.0,0.950,0.207,0.031,0.006
4,강로한,0.341456,12.0,0.911,2.177,0.249,0.068
...,...,...,...,...,...,...,...
603,황재균,-0.418952,1.0,0.968,-2.839,-0.324,-0.089
604,황재균,2.913118,10.0,0.970,11.303,1.695,0.350
605,황진수,-0.613018,0.0,0.000,-0.000,-0.000,-0.000
606,황진수,-0.582516,0.0,1.000,-2.330,-0.350,-0.072


In [47]:
## 선수 한 명의 기록 합친 값이 들어갈 df 생성
dfn_1_uniqs = pd.DataFrame(columns = dfn_1_uniq.columns)
dfn_1_uniqs['선수명'] = dfn_1_uniq['선수명'].unique()
dfn_1_uniqs

,선수명,IP,E,FPCT,March_score,kWAR_score,AHP_score
0,강경학,NaN,NaN,NaN,NaN,NaN,NaN
1,강로한,NaN,NaN,NaN,NaN,NaN,NaN
2,강민국,NaN,NaN,NaN,NaN,NaN,NaN
3,강민호,NaN,NaN,NaN,NaN,NaN,NaN
4,강백호,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
277,황대인,NaN,NaN,NaN,NaN,NaN,NaN
278,황윤호,NaN,NaN,NaN,NaN,NaN,NaN
279,황재균,NaN,NaN,NaN,NaN,NaN,NaN
280,황진수,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
## 선수 한 명의 1년치 기록 합치기
ID_1 = '강경학'
row = 0
for d in range(1, len(dfn_1_uniqs.columns)) :
    c = 0
    row = 0
    for a in range(0, len(dfn_1_uniq)) :
        ID = dfn_1_uniq.iloc[a, 0]
        if ID == ID_1 :
            c += dfn_1_uniq.iloc[a, d]

        else :
            dfn_1_uniqs.iloc[row, d] = c
            row += 1
            c = 0
            c += dfn_1_uniq.iloc[a, d]
            ID_1 = dfn_1_uniq.iloc[a, 0]
dfn_1_uniqs

,선수명,IP,E,FPCT,March_score,kWAR_score,AHP_score
0,강경학,-0.850417,0,0,0,0,0
1,강로한,0.748501,4.0,2.936,-3.614,-0.294,-0.072
2,강민국,-0.950489,20.0,2.834,4.442,0.486,0.117
3,강민호,2.466401,9.0,2.734,-4.128,-0.502,-0.102
4,강백호,-0.803455,8.0,0.989,19.514,2.439,0.903
...,...,...,...,...,...,...,...
277,황대인,-1.057601,0.0,3.0,-4.434,-0.445,-0.078
278,황윤호,-0.760085,0.0,2.0,-3.333,-0.366,-0.086
279,황재균,1.909233,6.0,2.9,-4.56,-0.546,-0.139
280,황진수,-1.195534,11.0,2.938,7.294,1.371,0.248


In [49]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 첫 번째 칼럼도 하나씩 밀어줌
IP_list = dfn_1_uniqs['IP'].to_list()
IP_list.insert(0, 0)
IP_list = IP_list[:-1]
dfn_1_uniqs['IP'] = IP_list

In [50]:
## 두 번째칼럼부터 하나씩 밀려서 들어가서 두 번째 칼럼도 하나씩 밀어줌
ID_list = dfn_1_uniqs['선수명'].to_list()
ID_list.insert(0, 0)
ID_list = ID_list[:-1]
dfn_1_uniqs['선수명'] = ID_list

In [51]:
## 그리고 나서 첫 번째 쓸모없는 row 삭제
dfn_1_uniqs = dfn_1_uniqs.drop([0])
dfn_1_uniqs = dfn_1_uniqs.reset_index(drop = True)
dfn_1_uniqs

,선수명,IP,E,FPCT,March_score,kWAR_score,AHP_score
0,강경학,-0.850417,4.0,2.936,-3.614,-0.294,-0.072
1,강로한,0.748501,20.0,2.834,4.442,0.486,0.117
2,강민국,-0.950489,9.0,2.734,-4.128,-0.502,-0.102
3,강민호,2.466401,8.0,0.989,19.514,2.439,0.903
4,강백호,-0.803455,3.0,3.978,-4.16,-0.517,-0.219
...,...,...,...,...,...,...,...
276,홍창기,-1.659633,0.0,3.0,-4.434,-0.445,-0.078
277,황대인,-1.057601,0.0,2.0,-3.333,-0.366,-0.086
278,황윤호,-0.760085,6.0,2.9,-4.56,-0.546,-0.139
279,황재균,1.909233,11.0,2.938,7.294,1.371,0.248


In [52]:
dfn_1_uniqs.index = dfn_1_uniqs['선수명']
del dfn_1_uniqs['선수명'], dfn_1_uniqs['FPCT']
dfn_1_uniqs

,IP,E,March_score,kWAR_score,AHP_score
선수명,,,,,
강경학,-0.850417,4.0,-3.614,-0.294,-0.072
강로한,0.748501,20.0,4.442,0.486,0.117
강민국,-0.950489,9.0,-4.128,-0.502,-0.102
강민호,2.466401,8.0,19.514,2.439,0.903
강백호,-0.803455,3.0,-4.16,-0.517,-0.219
...,...,...,...,...,...
홍창기,-1.659633,0.0,-4.434,-0.445,-0.078
황대인,-1.057601,0.0,-3.333,-0.366,-0.086
황윤호,-0.760085,6.0,-4.56,-0.546,-0.139


In [53]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율\전처리')
dfn_1_uniqs.to_csv('2019_전포지션_수비점수_March_kWAR_AHP_이닝표준화.csv', encoding = 'cp949')